In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector
import traceback

def recommend_recipe_sites(user_contents, num):
    ret = None
    conn = None
    try:
        cfg = {
            "host": "35.229.201.139",
            "port": 3306,
            "database": "icook_db",
            "user": "iii",
            "password": "Qwer_0987"
        }
        conn = mysql.connector.connect(**cfg)
        df = pd.read_sql("select * from final_result", conn)
        train_series = df["tags"] # Series
        test_series = pd.Series(user_contents) # Series
#         tokenizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), min_df=0, stop_words="english")
        tokenizer = TfidfVectorizer()
        train_tfidf = tokenizer.fit_transform(train_series)
        test_tfidf = tokenizer.transform(test_series)
        df_cs = pd.DataFrame(cosine_similarity(test_tfidf, train_tfidf), columns=train_series.index)
        df_cs["ix_test"] = test_series.index
        # pd.melt(): http://shzhangji.com/cnblogs/2017/09/30/pandas-and-tidy-data/
        # frame: 需要處理的數據框；
        # id_vars: 保持原樣的數據列；
        # value_vars: 需要被轉換成變量值的數據列；
        # var_name: 轉換後變量的列名；
        # value_name: 數值變量的列名。
        score = pd.melt(
            df_cs,
            id_vars = "ix_test",
            var_name = "ix_train",
            value_name = "score"
        )
        df["score"] = score["score"]
        # score遞增排序並取出指定的筆數
#         df = df.sort_values(by="score")[:-(num+1):-1]
        df = df.sort_values(by=["score" ], ascending=[False])
        ret = df[["name", "tags", "score","url"]].head(num)
    except Exception as e:
        print("Error: {}".format(e))
        traceback.print_exc()
    finally:
        if conn:
            conn.close()
    return ret
    
recommend_result = recommend_recipe_sites(user_contents="豬肉 牛奶", num=5)
print(recommend_result)

             name      tags     score                               url
285  日式漢堡排 {露營ok}  豬肉 洋蔥 牛奶  0.861789  https://icook.tw//recipes/153137
394  自製鬆餅粉 {露營ok}        牛奶  0.831482   https://icook.tw//recipes/98719
295  法式可麗餅 {露營OK}        牛奶  0.831482  https://icook.tw//recipes/148323
347          香Q蛋餅        牛奶  0.831482  https://icook.tw//recipes/172307
325  日式可麗餅 {露營OK}        牛奶  0.831482  https://icook.tw//recipes/125205
